<a href="https://colab.research.google.com/github/zion645/nano_gtp/blob/master/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#download dataset that were training on
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2023-04-27 22:53:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-04-27 22:53:45 (20.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
#read the data and inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print('length if dataset in number of characters: ', len(text))

length if dataset in number of characters:  1115394


In [ ]:
#lets see how the first 1000 characters look like
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
#now were going to get all the unique characters, by first wrapping the text into
#a set, which will get wrapt into a list and sorted

chars = sorted(list(set(text)))
vocab_size = len(chars) #the possible elements/characters of our sequences that the model sees
print(''.join(chars))
print(vocab_size)




 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
#now the strategy to tokenize the input text
#create a mapping from characters to integers (tokenizing)
stoi = {ch:i for i, ch in enumerate(chars)} #enumerate fnt creates an iterable that generates pairs of (index, character) for each character in the string 'chars'
                                            #it unpacks each pair into variables (index, character)
itos = {i:ch for i, ch in enumerate(chars)} #key value pairs are switched
encode = lambda s: [stoi[c] for c in s] #ecoder= take a string, output a list of integers
decode = lambda l: ''.join([ itos[i] for i in l ]) #decoder= take a list of integers, output a string

print('encoded message message:', encode('hii there'))
print('decoded message: ', decode(encode('hii there')))

encoded message message: [46, 47, 47, 1, 58, 46, 43, 56, 43]
decoded message:  hii there


In [ ]:
#let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use pytorch
data = torch.tensor(encode(text), dtype=torch.long) # use the entire text, original text
print(data.shape, data.dtype)
print(data[:1000]) #all this characters is the same as the previous text we saw

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
#split the data into test/train
# this will help us understand to what extend the model is overfit
n = int(0.9*len(data)) #the first 90% will be train data, rest validation
train_data = data[:n]
val_data = data[n:]

In [ ]:
#block size is the amount of characters we feed to the model
# we want to give the model blocks that are small, to look at everything in between, and big blocks
block_size =8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# illustration for the previous sniped
# here we can see that the transformer is able to see one to many characters
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
# this is the batch of input that will be fed into our transformer
torch.manual_seed(1337) #this seed is a random number
batch_size = 4 #how many independent sequences will we process in parallel?
block_size = 8 #what is the maximum context length for predictions?

def get_batch(split):
  #generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): #batch dimensions
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")



inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [ ]:
#our input to the transformer
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
#now were going to start with NN, the bigram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    #each token directly reads off the logits for the next token from a lookup table
    #create the token embedding table, which is of size vocab_size by vocab_size
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    #idx and targets are both (B, T) tensor of integers
    # so when we pass idx, every integer is going to look at our embedding table, and plugout that index number
    logits = self.token_embedding_table(idx) #(torch is going to arrange the embedding table by - Batch x Time x Channel(vocab_size))

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape #define our multidimensional array and then make it into 2
      logits = logits.view(B*T, C) # redhape into a 2d array to preserve the 2 channel dimension for Torch
      targets = targets.view(B*T)

    #now evaluate the loss-function which is cross-entropy in torch
    #this measures the quality of logits with respect to the targets,
    #meaning how well are we predicting the next character based on the logits
    #the correct number of logits should be a high number
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  # generate function for the model
  # we take the same input idx
  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx) #because this returns to our Forward function, we have to make sure our target to be optional
      #focus only on the last time step
      logits = logits[:,-1, :] #becomes (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) #(B, C)
      #sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      #concat sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


m=BigramLanguageModel(vocab_size)
logits, loss= m(xb, yb) #passing the inputs and target
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
#now train the model

#create a pytorch optimizer
#this will take the parameters and update the gradients
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):
  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  print(loss.item())


Streaming output truncated to the last 5000 lines.
2.5105183124542236
2.498452663421631
2.6411519050598145
2.545017957687378
2.5822653770446777
2.5510268211364746
2.5688059329986572
2.547714948654175
2.502406358718872
2.4768576622009277
2.514636993408203
2.4804952144622803
2.5187432765960693
2.471015214920044
2.5060224533081055
2.553105115890503
2.6693813800811768
2.6579267978668213
2.5331437587738037
2.5448012351989746
2.519066572189331
2.494197368621826
2.6255569458007812
2.6027286052703857
2.493701219558716
2.585630178451538
2.568652629852295
2.54506254196167
2.5943119525909424
2.4979991912841797
2.463721513748169
2.5440189838409424
2.6040382385253906
2.538984537124634
2.613231897354126
2.6427505016326904
2.4055047035217285
2.4835338592529297
2.556751251220703
2.57570219039917
2.6242897510528564
2.5811281204223633
2.531205892562866
2.5506672859191895
2.49810791015625
2.5038766860961914
2.588451623916626
2.5774757862091064
2.493567705154419
2.5814247131347656
2.589650869369507
2.5571

In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


#Up to this point the tokens are not talking to each other

#Now is time for our tokens to talk to each other


#The mathematical trick in Self-attention

In [ ]:
#consider the following toy example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In this example we 8 tokens that are not currently talking to each other, but we would like them to.
We want to couple them so that 5th token only talks to 4th, 3rd, 2nd, and 1st. Given that if it were to talk to 6th and up, these would be future tokens. Information only flows from the past, and were going to try to predict the future.

If Im the 5th token, and i want to connect to the past, is to do an average of all the preceding elements. This way it (this feature vector) would summarize the present with the context of my history.

For every batch element independently(for every element in that sequence), we want to calculate the average of all the vectors in all the previous tokens and also at the present token.

In [ ]:
# Here were calculating every batch element to average all previous vectors up to the present
# we want x[b,t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t, C)
    xbow[b,t] = torch.mean(xprev, 0)

xbow = x bag of words
here were initializing at zero. (Not being efficient yet)
then were averaging and iterating over batch(B) and time(T)
xprev = are at the batch dimension and up to t-token and becomes of shape t, C
xbow = then were averaging the time(T) and storing C into xbow

In [ ]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [ ]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

the reason we see the first set of numbers equal is because xbow is the average of the the previous number and given its the first set, should be the same.
however in xbow, the second pair is the average of the 1st, and 2nd batch.

This for-loop is not efficient, the better way is to do matrix multiplication.

In [ ]:
# same example as the toy-example above

torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b #matrix multiplication a * b = c (dot product)
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


the result is the dot product of matrix a and b

In [ ]:
#now were going to apply linear algebra
# this would be our a example from above

wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
# b example
xbow2 = wei @ x #it will give us a (B,T,T)@(B,T,C) -----> (B,T,C)


In [ ]:
torch.allclose(xbow, xbow2)

True

In [ ]:
# another way to write the above - using Softmax function
#
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # the amount of tokens from the past we want to aggregate. Although this points are set to 0, they are data dependent and will look at each other
wei = wei.masked_fill(tril == 0, float('-inf')) #for all the numbers that 0 in tril, make them negative infinity. Tokens from the past cannot communicate by setting them to -inf
wei = F.softmax(wei, dim=-1) #taking a softmax of every single row, is to normalize, exponentiate and divide by the sum
xbow3 = wei @ x # the matrix multiplication
torch.allclose(xbow, xbow3)

True

we can do weighted aggregations from the past elements by using matrix multiplication.


In [ ]:
# self-attention block

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)

#lets see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T)) # the amount of tokens from the past we want to aggregate. Although this points are set to 0, they are data dependent and will look at each other
wei = wei.masked_fill(tril == 0, float('-inf')) #for all the numbers that 0 in tril, make them negative infinity. Tokens from the past cannot communicate by setting them to -inf
wei = F.softmax(wei, dim=-1) #taking a softmax of every single row, is to normalize, exponentiate and divide by the sum

v = value(x)
out = wei @ x

out.shape

Attention is a communication mechanism. Can be seen as nodes in a directed graph, looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data dependent weights.

There's no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.

Each example across batch dimension is of course processed completely independently and necer talk to each other.

In an encoder

Multiheaded attention:
Applying multiple heads in parallel